# DDOS ATTACK USING MACHINE LEARNING

- Dataset [https://www.kaggle.com/jacobvs/ddos-attack-network-logs?select=final-dataset.arff]

- 

### Importing Required Modules 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,confusion_matrix

### Load the dataset 

In [2]:
data=pd.read_csv("final-dataset.csv")
data.head()

C:\Users\sravani\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,SRC_ADD,DES_ADD,PKT_ID,FROM_NODE,TO_NODE,PKT_TYPE,PKT_SIZE,FLAGS,FID,SEQ_NUMBER,...,PKT_RATE,BYTE_RATE,PKT_AVG_SIZE,UTILIZATION,PKT_DELAY,PKT_SEND_TIME,PKT_RESEVED_TIME,FIRST_PKT_SENT,LAST_PKT_RESEVED,PKT_CLASS
0,3.00,24.30,389693,21,23,tcp,1540,-------,4,11339,...,328.241,505490.0,1540.0,0.236321,0.0,35.519662,35.550032,1.000000,50.021920,Normal
1,15.00,24.15,201196,23,24,tcp,1540,-------,16,6274,...,328.206,505437.0,1540.0,0.236337,0.0,20.156478,20.186848,1.000000,50.030211,Normal
2,24.15,15.00,61905,23,22,ack,55,-------,16,1930,...,328.206,18051.3,55.0,0.008441,0.0,7.039952,7.069962,1.030045,50.060221,UDP-Flood
3,24.90,9.00,443135,23,21,ack,55,-------,10,12670,...,328.064,18043.5,55.0,0.008437,0.0,39.617967,39.647976,1.030058,50.060098,Normal
4,24.80,8.00,157335,23,21,ack,55,-------,9,4901,...,328.114,18046.2,55.0,0.008438,0.0,16.029803,16.059813,1.030054,50.061864,Normal


- label={'SRC_ADD','DES_ADD','PKT_ID','FROM_NODE','TO_NODE','PKT_TYPE','PKT_SIZE','FLAGS','FID','SEQ_NUMBER...','PKT_RATE','BYTE_RATE','PKT_AVG_SIZE','UTILIZATION','PKT_DELAY','PKT_SEND_TIME','PKT_RESEVED_TIME','FIRST_PKT_SENT','LAST_PKT_RESEVED','PKT_CLASS'}

### Reading data

In [3]:
data.shape

(977025, 28)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 977025 entries, 0 to 977024
Data columns (total 28 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   SRC_ADD           977025 non-null  float64
 1   DES_ADD           977025 non-null  float64
 2   PKT_ID            977025 non-null  int64  
 3   FROM_NODE         977025 non-null  int64  
 4   TO_NODE           977025 non-null  int64  
 5   PKT_TYPE          977025 non-null  object 
 6   PKT_SIZE          977025 non-null  int64  
 7   FLAGS             977025 non-null  object 
 8   FID               977025 non-null  int64  
 9   SEQ_NUMBER        977025 non-null  int64  
 10  NUMBER_OF_PKT     977025 non-null  int64  
 11  NUMBER_OF_BYTE    977025 non-null  int64  
 12  NODE_NAME_FROM    977025 non-null  object 
 13  NODE_NAME_TO      977024 non-null  object 
 14  PKT_IN            977023 non-null  float64
 15  PKT_OUT           977023 non-null  float64
 16  PKT_R             97

In [5]:
data.describe()

,SRC_ADD,DES_ADD,PKT_ID,FROM_NODE,TO_NODE,PKT_SIZE,FID,SEQ_NUMBER,NUMBER_OF_PKT,NUMBER_OF_BYTE,...,PKT_R,PKT_DELAY_NODE,BYTE_RATE,PKT_AVG_SIZE,UTILIZATION,PKT_DELAY,PKT_SEND_TIME,PKT_RESEVED_TIME,FIRST_PKT_SENT,LAST_PKT_RESEVED
count,977025.000000,977025.000000,977025.000000,977025.000000,977025.000000,977025.000000,977025.000000,9.770250e+05,977025.000000,9.770250e+05,...,977023.000000,977023.000000,9.770220e+05,977022.000000,977022.000000,977022.000000,977022.000000,977022.000000,977022.000000,977022.000000
mean,14.460194,18.243119,290238.604907,19.589946,20.829160,1054.529304,13.933393,1.853144e+04,13472.640645,1.139637e+07,...,31.354636,0.001084,3.319479e+05,1054.528603,0.108684,0.005852,31.332666,31.365501,5.171337,50.683292
std,8.769697,8.542978,195616.043087,6.281343,5.282073,3321.060303,10.974616,1.375221e+05,4264.981917,1.054476e+07,...,17.954607,0.051374,3.989866e+05,3321.072866,0.100563,0.019870,17.953541,17.955156,9.081897,16.599191
min,0.000000,0.000000,0.000000,0.000000,0.000000,55.000000,0.000000,-1.000000e+00,9.000000,6.050000e+02,...,0.015243,-0.039216,5.378760e+01,55.000000,0.000006,-0.114464,0.000000,0.030015,0.000000,1.756152
25%,7.000000,11.000000,101227.000000,21.000000,21.000000,55.000000,5.000000,2.354000e+03,9108.000000,8.850600e+05,...,16.672206,0.000000,1.805740e+04,55.000000,0.008441,0.000000,16.651158,16.682931,1.000000,50.021920
50%,14.000000,24.130000,281220.000000,22.000000,23.000000,1000.000000,11.000000,5.552000e+03,16091.000000,6.250000e+06,...,30.936930,0.000000,1.249430e+05,1000.000000,0.059605,0.000000,30.917858,30.949170,1.030016,50.040685
75%,24.130000,24.370000,461656.000000,23.000000,23.000000,1540.000000,21.000000,1.074900e+04,16092.000000,2.478010e+07,...,43.808412,0.000000,5.055260e+05,1540.000000,0.236321,0.000844,43.786495,43.818602,1.030051,50.060098
max,26.000000,26.000000,641599.000000,26.000000,26.000000,65535.000000,40.000000,3.015537e+06,16103.000000,2.479860e+07,...,75.999821,50.025737,1.677420e+06,65535.200000,0.236498,0.153680,75.983741,75.999821,25.000000,75.999821


In [6]:
data.columns

Index(['SRC_ADD', 'DES_ADD', 'PKT_ID', 'FROM_NODE', 'TO_NODE', 'PKT_TYPE',
       'PKT_SIZE', 'FLAGS', 'FID', 'SEQ_NUMBER', 'NUMBER_OF_PKT',
       'NUMBER_OF_BYTE', 'NODE_NAME_FROM', 'NODE_NAME_TO', 'PKT_IN', 'PKT_OUT',
       'PKT_R', 'PKT_DELAY_NODE', 'PKT_RATE', 'BYTE_RATE', 'PKT_AVG_SIZE',
       'UTILIZATION', 'PKT_DELAY', 'PKT_SEND_TIME', 'PKT_RESEVED_TIME',
       'FIRST_PKT_SENT', 'LAST_PKT_RESEVED', 'PKT_CLASS'],
      dtype='object')

### Preprocessing

In [7]:
missing_values=data.isnull()
missing_values

,SRC_ADD,DES_ADD,PKT_ID,FROM_NODE,TO_NODE,PKT_TYPE,PKT_SIZE,FLAGS,FID,SEQ_NUMBER,...,PKT_RATE,BYTE_RATE,PKT_AVG_SIZE,UTILIZATION,PKT_DELAY,PKT_SEND_TIME,PKT_RESEVED_TIME,FIRST_PKT_SENT,LAST_PKT_RESEVED,PKT_CLASS
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977020,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
977021,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
977022,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
977023,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [8]:
# count the number of null values
count_values=data.isnull().sum()
count_values

SRC_ADD             0
DES_ADD             0
PKT_ID              0
FROM_NODE           0
TO_NODE             0
PKT_TYPE            0
PKT_SIZE            0
FLAGS               0
FID                 0
SEQ_NUMBER          0
NUMBER_OF_PKT       0
NUMBER_OF_BYTE      0
NODE_NAME_FROM      0
NODE_NAME_TO        1
PKT_IN              2
PKT_OUT             2
PKT_R               2
PKT_DELAY_NODE      2
PKT_RATE            2
BYTE_RATE           3
PKT_AVG_SIZE        3
UTILIZATION         3
PKT_DELAY           3
PKT_SEND_TIME       3
PKT_RESEVED_TIME    3
FIRST_PKT_SENT      3
LAST_PKT_RESEVED    3
PKT_CLASS           3
dtype: int64

In [9]:
ndata=data.dropna()
ndata.isnull().sum()

SRC_ADD             0
DES_ADD             0
PKT_ID              0
FROM_NODE           0
TO_NODE             0
PKT_TYPE            0
PKT_SIZE            0
FLAGS               0
FID                 0
SEQ_NUMBER          0
NUMBER_OF_PKT       0
NUMBER_OF_BYTE      0
NODE_NAME_FROM      0
NODE_NAME_TO        0
PKT_IN              0
PKT_OUT             0
PKT_R               0
PKT_DELAY_NODE      0
PKT_RATE            0
BYTE_RATE           0
PKT_AVG_SIZE        0
UTILIZATION         0
PKT_DELAY           0
PKT_SEND_TIME       0
PKT_RESEVED_TIME    0
FIRST_PKT_SENT      0
LAST_PKT_RESEVED    0
PKT_CLASS           0
dtype: int64

### Creating dataframe for selected attributes

In [10]:
df=pd.DataFrame(data,columns=['SRC_ADD','DES_ADD','PKT_ID','FROM_NODE','TO_NODE','PKT_TYPE','PKT_SIZE','FLAGS','FID','SEQ_NUMBER...','PKT_RATE','BYTE_RATE','PKT_AVG_SIZE','UTILIZATION','PKT_DELAY','PKT_SEND_TIME','PKT_RESEVED_TIME','FIRST_PKT_SENT','LAST_PKT_RESEVED','PKT_CLASS'])
df=df.sample(n=3000)
df

,SRC_ADD,DES_ADD,PKT_ID,FROM_NODE,TO_NODE,PKT_TYPE,PKT_SIZE,FLAGS,FID,SEQ_NUMBER...,PKT_RATE,BYTE_RATE,PKT_AVG_SIZE,UTILIZATION,PKT_DELAY,PKT_SEND_TIME,PKT_RESEVED_TIME,FIRST_PKT_SENT,LAST_PKT_RESEVED,PKT_CLASS
965360,4.10,24.24,364657,23,24,cbr,1000,-------,25,NaN,124.943,124943.0,1000.0,0.059605,0.000768,33.600000,33.631008,25.000000,75.022880,Normal
901023,24.20,2.00,361457,21,2,ack,55,-------,3,NaN,328.264,18054.5,55.0,0.008440,0.000000,33.353073,33.383082,1.030022,50.048477,Normal
790092,24.15,15.00,105961,23,22,ack,55,-------,16,NaN,328.206,18051.3,55.0,0.008441,0.000000,11.189978,11.219987,1.030045,50.060221,Normal
576464,24.40,4.00,569978,23,21,ack,55,-------,5,NaN,328.218,18052.0,55.0,0.008440,0.000000,49.346562,49.376571,1.030035,50.055377,Normal
231338,5.20,24.47,36877,21,23,cbr,1500,-------,26,NaN,1016.52,1524780.0,1500.0,0.130291,0.033240,35.377143,35.410383,1.000000,9.959955,UDP-Flood
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33699,1.00,24.10,110012,1,21,tcp,1540,-------,2,NaN,328.298,505578.0,1540.0,0.236321,0.000000,11.571544,11.601914,1.000000,50.013418,Normal
256681,3.00,24.30,292032,21,23,tcp,1540,-------,4,NaN,328.241,505490.0,1540.0,0.236321,0.000123,28.027115,28.057608,1.000000,50.021920,Normal
380096,3.00,24.30,493678,23,24,tcp,1540,-------,4,NaN,328.241,505490.0,1540.0,0.236321,0.001601,43.496208,43.528178,1.000000,50.021920,Normal
76333,4.00,24.40,135742,23,24,tcp,1540,-------,5,NaN,328.218,505455.0,1540.0,0.236321,0.000123,13.993981,14.024474,1.000000,50.025368,Normal


n_sample=3000
classes_per_sample=int(np.ceil(n_sample/len(label)))
x_sample=None
for l in label:
    if x_sample is None:
        x_sample=x[x['PKT_CLASS']==l].sample(n=classes_per_sample)
    else:
        x_sample=pd.concat([x_sample,x[x['PKT_CLASS']==l].sample(n=classes_per_sample)],axis=0)

In [11]:
df['PKT_CLASS'].value_counts()

Normal        2678
UDP-Flood      287
Smurf           20
SIDDOS          11
HTTP-FLOOD       4
Name: PKT_CLASS, dtype: int64

Y = []
for vals in y:
    Y.append(np.where(label=vals)[0][0])
print(len(Y))

### Selecting Features and Target

    - Features : SRC_ADD	DES_ADD	PKT_ID	FROM_NODE	TO_NODE	PKT_TYPE	PKT_SIZE	FLAGS	FID	SEQ_NUMBER...	PKT_RATE	BYTE_RATE	PKT_AVG_SIZE	UTILIZATION	PKT_DELAY	PKT_SEND_TIME	PKT_RESEVED_TIME	FIRST_PKT_SENT	LAST_PKT_RESEVED

    - Target : PKT_CLASS

In [12]:
X=df.iloc[0:26]

In [13]:
X=df.drop(['PKT_CLASS'],axis=1)

In [14]:
X

,SRC_ADD,DES_ADD,PKT_ID,FROM_NODE,TO_NODE,PKT_TYPE,PKT_SIZE,FLAGS,FID,SEQ_NUMBER...,PKT_RATE,BYTE_RATE,PKT_AVG_SIZE,UTILIZATION,PKT_DELAY,PKT_SEND_TIME,PKT_RESEVED_TIME,FIRST_PKT_SENT,LAST_PKT_RESEVED
965360,4.10,24.24,364657,23,24,cbr,1000,-------,25,NaN,124.943,124943.0,1000.0,0.059605,0.000768,33.600000,33.631008,25.000000,75.022880
901023,24.20,2.00,361457,21,2,ack,55,-------,3,NaN,328.264,18054.5,55.0,0.008440,0.000000,33.353073,33.383082,1.030022,50.048477
790092,24.15,15.00,105961,23,22,ack,55,-------,16,NaN,328.206,18051.3,55.0,0.008441,0.000000,11.189978,11.219987,1.030045,50.060221
576464,24.40,4.00,569978,23,21,ack,55,-------,5,NaN,328.218,18052.0,55.0,0.008440,0.000000,49.346562,49.376571,1.030035,50.055377
231338,5.20,24.47,36877,21,23,cbr,1500,-------,26,NaN,1016.52,1524780.0,1500.0,0.130291,0.033240,35.377143,35.410383,1.000000,9.959955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33699,1.00,24.10,110012,1,21,tcp,1540,-------,2,NaN,328.298,505578.0,1540.0,0.236321,0.000000,11.571544,11.601914,1.000000,50.013418
256681,3.00,24.30,292032,21,23,tcp,1540,-------,4,NaN,328.241,505490.0,1540.0,0.236321,0.000123,28.027115,28.057608,1.000000,50.021920
380096,3.00,24.30,493678,23,24,tcp,1540,-------,4,NaN,328.241,505490.0,1540.0,0.236321,0.001601,43.496208,43.528178,1.000000,50.021920
76333,4.00,24.40,135742,23,24,tcp,1540,-------,5,NaN,328.218,505455.0,1540.0,0.236321,0.000123,13.993981,14.024474,1.000000,50.025368


In [15]:
Y=df['PKT_CLASS']

In [16]:
Y

965360       Normal
901023       Normal
790092       Normal
576464       Normal
231338    UDP-Flood
            ...    
33699        Normal
256681       Normal
380096       Normal
76333        Normal
660989       Normal
Name: PKT_CLASS, Length: 3000, dtype: object

### Model Selection

### Training and Testing

In [17]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,stratify=Y,test_size=0.20,random_state=0)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000 entries, 965360 to 660989
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   SRC_ADD           3000 non-null   float64
 1   DES_ADD           3000 non-null   float64
 2   PKT_ID            3000 non-null   int64  
 3   FROM_NODE         3000 non-null   int64  
 4   TO_NODE           3000 non-null   int64  
 5   PKT_TYPE          3000 non-null   object 
 6   PKT_SIZE          3000 non-null   int64  
 7   FLAGS             3000 non-null   object 
 8   FID               3000 non-null   int64  
 9   SEQ_NUMBER...     0 non-null      float64
 10  PKT_RATE          3000 non-null   object 
 11  BYTE_RATE         3000 non-null   float64
 12  PKT_AVG_SIZE      3000 non-null   float64
 13  UTILIZATION       3000 non-null   float64
 14  PKT_DELAY         3000 non-null   float64
 15  PKT_SEND_TIME     3000 non-null   float64
 16  PKT_RESEVED_TIME  3000 non-null   f

### Label Encoding

In [19]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [20]:
df['PKT_TYPE'] = le.fit_transform(df['PKT_TYPE'])

In [21]:
df['PKT_TYPE']

965360    1
901023    0
790092    0
576464    0
231338    1
         ..
33699     3
256681    3
380096    3
76333     3
660989    0
Name: PKT_TYPE, Length: 3000, dtype: int32

In [22]:
df['PKT_TYPE'].unique()

array([1, 0, 3, 2])

In [23]:
df['FLAGS']

965360    -------
901023    -------
790092    -------
576464    -------
231338    -------
           ...   
33699     -------
256681    -------
380096    -------
76333     -------
660989    -------
Name: FLAGS, Length: 3000, dtype: object

In [24]:
df['FLAGS'] = le.fit_transform(df['FLAGS'])

In [25]:
df['FLAGS'].unique()

array([0, 1])

In [26]:
df['PKT_CLASS']

965360       Normal
901023       Normal
790092       Normal
576464       Normal
231338    UDP-Flood
            ...    
33699        Normal
256681       Normal
380096       Normal
76333        Normal
660989       Normal
Name: PKT_CLASS, Length: 3000, dtype: object

In [27]:
df['PKT_CLASS'] = le.fit_transform(df['PKT_CLASS'])

In [28]:
df['PKT_CLASS'].unique()

array([1, 4, 3, 2, 0])

In [29]:
df['PKT_RATE']

965360    124.943
901023    328.264
790092    328.206
576464    328.218
231338    1016.52
           ...   
33699     328.298
256681    328.241
380096    328.241
76333     328.218
660989    328.432
Name: PKT_RATE, Length: 3000, dtype: object

In [30]:
df.PKT_RATE.unique()

array([124.942826, 328.26412, 328.20604199999997, 328.218006,
       1016.5229619999999, 328.26403999999997, 328.297902, 124.943426,
       '328.491489', 328.217832, 328.491489, 328.24105099999997,
       124.940628, 328.491596, 124.943226, 328.316229, 124.942227,
       328.522879, 328.19309, 124.942027, 328.204637, 328.167486,
       328.29785499999997, 328.113525, 328.16776699999997, 328.192835,
       328.43200099999996, '328.206042', '328.240918', 328.561918,
       962.684973, 328.205808, 1016.5429300000001, 328.46027799999996,
       328.460131, 328.11323, 1016.5167230000001, 328.064183, 124.942626,
       23.255773, 328.240918, 328.431807, 124.944425, 328.52294700000004,
       328.20485099999996, 124.941827, 1066.08599, 1016.4372,
       328.06386200000003, 1016.496869, 1016.5491699999999,
       124.94122800000001, 23.215379000000002, 124.944025, '328.491596',
       '328.26404', 124.943825, 95.145988, 124.941627, 328.561898,
       '328.064183', 124.943026, 23.255775, 124.94

In [31]:
df['PKT_RATE']=df['PKT_RATE'].astype(float)
df['PKT_CLASS']=df['PKT_CLASS'].astype(float)

In [32]:
df['PKT_CLASS']

965360    1.0
901023    1.0
790092    1.0
576464    1.0
231338    4.0
         ... 
33699     1.0
256681    1.0
380096    1.0
76333     1.0
660989    1.0
Name: PKT_CLASS, Length: 3000, dtype: float64

In [33]:
df['PKT_TYPE']=df['PKT_TYPE'].astype(float)
df['PKT_TYPE']

965360    1.0
901023    0.0
790092    0.0
576464    0.0
231338    1.0
         ... 
33699     3.0
256681    3.0
380096    3.0
76333     3.0
660989    0.0
Name: PKT_TYPE, Length: 3000, dtype: float64

In [34]:
df['FLAGS']=df['FLAGS'].astype(float)
df['FLAGS']

965360    0.0
901023    0.0
790092    0.0
576464    0.0
231338    0.0
         ... 
33699     0.0
256681    0.0
380096    0.0
76333     0.0
660989    0.0
Name: FLAGS, Length: 3000, dtype: float64

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000 entries, 965360 to 660989
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   SRC_ADD           3000 non-null   float64
 1   DES_ADD           3000 non-null   float64
 2   PKT_ID            3000 non-null   int64  
 3   FROM_NODE         3000 non-null   int64  
 4   TO_NODE           3000 non-null   int64  
 5   PKT_TYPE          3000 non-null   float64
 6   PKT_SIZE          3000 non-null   int64  
 7   FLAGS             3000 non-null   float64
 8   FID               3000 non-null   int64  
 9   SEQ_NUMBER...     0 non-null      float64
 10  PKT_RATE          3000 non-null   float64
 11  BYTE_RATE         3000 non-null   float64
 12  PKT_AVG_SIZE      3000 non-null   float64
 13  UTILIZATION       3000 non-null   float64
 14  PKT_DELAY         3000 non-null   float64
 15  PKT_SEND_TIME     3000 non-null   float64
 16  PKT_RESEVED_TIME  3000 non-null   f

from sklearn.neighbors import KNeighborsClassifier
classifier= KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, Y_train) 

In [36]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test =train_test_split(X,Y,stratify=Y,test_size=0.20,random_state=3)

In [37]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(2400, 19)
(2400,)
(600, 19)
(600,)


In [38]:
from sklearn import tree
clf_2 = tree.DecisionTreeClassifier(min_samples_split=2)
clf_2.fit(X_train,Y_train)
pred_2 = clf_2.predict(X_test)

ValueError: could not convert string to float: 'tcp'

In [41]:

# evaluate decision tree performance on train and test sets with different tree depths
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from matplotlib import pyplot
# create dataset
X=df.drop(['PKT_CLASS'],axis=1)
y=df['PKT_CLASS']
# split into train test sets
X_train,X_test,y_train,y_test = train_test_split(X,y,stratify=y,test_size=0.20,random_state=0)
# define lists to collect scores
train_scores, test_scores = list(), list()
# define the tree depths to evaluate
values = [i for i in range(1, 21)]
# evaluate a decision tree for each depth
for i in values:
	# configure the model
	model = DecisionTreeClassifier(max_depth=i)
	# fit model on the training dataset
	model.fit(X_train, y_train)
	# evaluate on the train dataset
	train_yhat = model.predict(X_train)
	train_acc = accuracy_score(y_train, train_yhat)
	train_scores.append(train_acc)
	# evaluate on the test dataset
	test_yhat = model.predict(X_test)
	test_acc = accuracy_score(y_test, test_yhat)
	test_scores.append(test_acc)
	# summarize progress
	print('>%d, train: %.3f, test: %.3f' % (i, train_acc, test_acc))
# plot of train and test scores vs tree depth
pyplot.plot(values, train_scores, '-o', label='Train')
pyplot.plot(values, test_scores, '-o', label='Test')
pyplot.legend()
pyplot.show()

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

print(X_train.shape)
scaler=StandardScaler()
x_train=scalar.fit_transform(X_train)
x_test=scalar.transform(X_test)
y_train=np.array(Y_train)
y_test=np.array(Y_test)


from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['PKT_TYPE'] = le.fit_transform(df['PKT_TYPE'])
df['FLAGS']=le.fit_transform(['FLAGS'])
df['PKT_RATE']=le.fit_transform(['PKT_RATE'])
df['PKT_CLASS']=le.fit_transform(['PKT_CLASS'])

df['PKT_TYPE']=df['PKT_TYPE'].astype(float)

df['FLAGS']

df['FLAGS']=df['FLAGS'].astype(float)

df['PKT_RATE']=df['PKT_RATE'].astype(float)

df['PKT_CLASS']=df['PKT_CLASS'].astype(float)

from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier(n_neighbors = 7).fit(X_train, Y_train) 